In [ ]:
%matplotlib inline
from env import ObstacleTowerEnv
from agent import Agent
from replay_memory import ReplayMemory
from utils import Trainer


env = ObstacleTowerEnv(action_size=54, history_length=16)
agent = Agent(env, atoms=51, v_min=-20.0, v_max=20.0, batch_size=64, multi_step=3,
              discount=0.99, norm_clip=10.0, lr=5e-4, adam_eps=1.5e-4, hidden_size=256, noisy_std=0.1)
mem = ReplayMemory(int(50e3), env.window, agent.discount, agent.n, priority_weight=0.4, priority_exponent=0.5)
trainer = Trainer(episodes=int(20e3), replay_frequency=6, reward_clip=2.0,
                  max_steps=int(1e6), learning_start_step=int(5e3), target_update = int(2e3))

trainer.train(env, agent, mem, notebook_file="qlearning", load_file="model_1596371695")

In [ ]:
trainer.save(agent)

In [ ]:
trainer.eval(env, agent)
env.close()